In [2]:
import os
import json

PATH = "/hpc/projects/group.califano/GLM/scGraphLLM/scripts/slurm_out"

for dr in os.listdir(PATH):
    print(dr)

slurm_out_16726641
.ARACNe3_cached


In [10]:
run = "16726641"

success_file = open(f"/hpc/projects/group.califano/GLM/scGraphLLM/scripts/slurm_out/slurm_out_{run}/check_out/success.txt", "r")
success_file = [dir[:-1] for dir in success_file]

PATH = "/hpc/projects/group.califano/GLM/data/cellxgene/data/complete_data"

cell_count = 0
for cell_type in success_file:
    for f in os.listdir(f"{PATH}/{cell_type}"):
        if "info" in f:
            with open(f'{PATH}/{cell_type}/{f}', 'r') as file:
                data = json.load(file)
        
    cell_count += data["metacells"]["n_cells"]

print(f"{cell_count} cells are in the dataset!")

2149185 cells are in the dataset!


In [10]:
PATH = "/hpc/projects/group.califano/GLM/data/cellxgene/data/complete_data"

failed = []
cell_count = 0
for cell_type in os.listdir(PATH):
    try:
        with open(f'{PATH}/{cell_type}/info.json', 'r') as file:
            data = json.load(file)
            
        cell_count += data['stats']["metacells"]["n_cells"]
    except:
        failed.append(cell_type)
    
print(f"\n{cell_count} cells are in the dataset!")
print(len(failed))


4448154 cells are in the dataset!
43


In [12]:
p = "/hpc/projects/group.califano/GLM/scGraphLLM/scripts/slurm_out/slurm_out_16703767/check_out/failed.txt"
file = open(p, 'r')
file_str = "".join(file)
file.close()

count = 0
for f in failed:
    if f in file_str:
        count += 1

count

37

## Before preprocessing

In [1]:
import os
import sys
import scanpy as sc
import anndata as ad
from tqdm import tqdm
import contextlib

In [18]:
p = "/hpc/projects/group.califano/GLM/data/cellxgene/data/cell_type_all/abnormal_cell/partitions/pan-cancer_0.h5ad"

adata = sc.read_h5ad(p)
adata.X.toarray().shape

(11410, 19221)

In [19]:
p = "/hpc/projects/group.califano/GLM/data/cellxgene/data/cell_type_all"

count = 0
for d in tqdm(os.listdir(p)):
    for h in os.listdir(f'{p}/{d}/partitions'):
        # Look through all the h5ad files and check number of cells (rows in GEMs)
        adata = sc.read_h5ad(f'{p}/{d}/partitions/{h}')
        count += adata.X.shape[0]
        del adata
        
count

  1%|▏         | 9/666 [00:06<08:15,  1.32it/s]


KeyboardInterrupt: 

174,872,658

In [6]:
run = "16703767"

success_file = open(f"/hpc/projects/group.califano/GLM/scGraphLLM/scripts/slurm_out/slurm_out_{run}/check_out/success.txt", "r")
success_file = [dir[:-1] for dir in success_file]

p = "/hpc/projects/group.califano/GLM/data/cellxgene/data/cell_type_all"
count = 0
for d in tqdm(success_file):
    for h in os.listdir(f'{p}/{d}/partitions'):
        # Look through all the h5ad files and check number of cells (rows in GEMs)
        adata = sc.read_h5ad(f'{p}/{d}/partitions/{h}')
        count += adata.X.shape[0]

count

100%|██████████| 261/261 [21:19<00:00,  4.90s/it]


113692215

113,692,215

Check number of cells in the successfully loaded cell-types (before processing) - does this match after processing?

## Filter corrupted files in cache

In [1]:
import os
import shutil
import torch

In [2]:
SOURCE_PATH = "/hpc/projects/group.califano/GLM/data/pilotdata_1024"

os.makedirs(f"{SOURCE_PATH}_filtered", exist_ok=True)
for SET in ["train", "valHG", "valSG"]:
    os.makedirs(f"{SOURCE_PATH}_filtered/{SET}", exist_ok=True)
    for dr in os.listdir(f"{SOURCE_PATH}/{SET}"):
        try:
            torch.load(f"{SOURCE_PATH}/{SET}/{dr}")
            shutil.copy(f"{SOURCE_PATH}/{SET}/{dr}", f"{SOURCE_PATH}_filtered/{SET}/{dr}")
        except:
            continue

In [3]:
p = "/hpc/projects/group.califano/GLM/data/pilotdata_1024_filtered/train/aracne_1024_0.pt"
data = torch.load(p)
data

Data(x=[1293, 2], edge_index=[2, 3248], edge_weight=[3248])

In [12]:
data.x

tensor([[ 9502, 19249],
        [ 3928, 19249],
        [ 2326, 19249],
        ...,
        [14958, 19249],
        [ 8995, 19249],
        [17559, 19249]])

## Truc

In [9]:
f0 = open("/hpc/projects/group.califano/GLM/types0.txt", 'r')
f1 = open("/hpc/projects/group.califano/GLM/types1.txt", 'r')

file0 = []
for row in f0:
    a, b = row.split()
    file0.append(a)
    file0.append(b)

file1 = []
for row in f1:
    a, b = row.split()
    file1.append(a)
    file1.append(b)
  
notin0 = []
for tp in file1:
    if tp not in file0:
        notin0.append(tp)

print(len(notin0))

notin1 = []
for tp in file0:
    if tp not in file1:
        notin1.append(tp)

print(len(notin1))

90
16


In [14]:
print("Not in original:")
print(notin0)

Not in original:
['abnormal_cell', 'absorptive_cell', 'l2_3-6_intratelencephalic_projecting_glutamatergic_neuron', 'l2_3_intratelencephalic_projecting_glutamatergic_neuron', 'l4_intratelencephalic_projecting_glutamatergic_neuron', 'l5_6_near-projecting_glutamatergic_neuron', 'activated_cd8-positive_alpha-beta_t_cell_human', 'adipocyte', 'lactocyte', 'airway_submucosal_gland_duct_basal_cell', 'luminal_adaptive_secretory_precursor_cell_of_mammary_gland', 'luminal_hormone-sensing_cell_of_mammary_gland', 'lung_interstitial_macrophage', 'lung_perichondrial_fibroblast', 'lung_resident_memory_cd4-positive_alpha-beta_t_cell', 'lung_resident_memory_cd8-positive_alpha-beta_t_cell', 'blood_cell', 'blood_vessel_smooth_muscle_cell', 'brain_vascular_cell', 'mature_astrocyte', 'mature_gamma-delta_t_cell', 'mature_microglial_cell', 'cd14-low_cd16-positive_monocyte', 'cd14-positive_cd16-negative_classical_monocyte', 'memory_regulatory_t_cell', 'cd34-positive_cd38-negative_hematopoietic_stem_cell', 'cd3

In [13]:
print("\nNot in new:")
notin1


Not in new:


['l2_3-6_intratelencephalic_projecting_glutamatergic_cortical_neuron',
 'animal_cell',
 'mammary_alveolar_cell',
 'native_cell',
 'neuronal_brush_cell',
 'neuron_associated_cell_sensu_vertebrata',
 'dopaminergic_neuron',
 'phagocyte',
 'primary_sensory_neuron',
 'purkinje_cell',
 'eukaryotic_cell',
 'fat_cell',
 'granule_cell',
 'somatic_cell',
 'stellate_neuron',
 'subcutaneous_fat_cell']

In [2]:
import os

t = len(os.listdir("/hpc/projects/group.califano/GLM/data/cxg_cache_2048/train"))
sg = len(os.listdir("/hpc/projects/group.califano/GLM/data/cxg_cache_2048/valSG"))
hog = len(os.listdir("/hpc/projects/group.califano/GLM/data/cxg_cache_2048/valHOG"))

print(t, sg, hog)
t + sg + hog

2476737 1064478 405631


3946846

In [ ]:
import os

# Specify the file path
file_path = "/hpc/projects/group.califano/GLM/data/cellxgene/data/complete_data_ranked_z_score"

# Check if the file exists before removing
if os.path.exists(file_path):
    os.remove(file_path)
    print(f"{file_path} has been removed.")
else:
    print(f"{file_path} does not exist.")